In [ ]:
!cp ./drive/My\ Drive/Colab\ Notebooks/BIGDATA-term-project/popular-classifier/data-classify.zip ./
!unzip data-classify.zip
!rm -rf ./data/popular/.DS_Store
!rm -rf ./data/unpopular/.DS_Store
!rm -rf __MACOSX

## 1. Load caption script files

In [234]:
from sklearn.datasets import load_files

DATA_PATH = './data'
data = load_files(DATA_PATH)
X, y = data.data, data.target

## 2. Define data preprocessing function

In [235]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def preprocess(X, y):
  docs = []
  stemmer = WordNetLemmatizer()

  # text preprocessing
  for i, doc_raw in enumerate(X):
    doc = doc_raw.decode('utf-8')
    doc = re.sub(r'\[.+\]', ' ', doc)
    doc = re.sub(r'\(.+\)', ' ', doc)
    doc = re.sub(r'\W', ' ', doc)
    doc = re.sub(r'\s+', ' ', doc, flags=re.I)

    doc = doc.lower()

    word_list = [word for word in doc.split() if len(word) > 2]

    lemma_word_list = []
    for word, tag in pos_tag(word_list):
      main_tag = tag[0].lower()
      lemma_word = ''

      if main_tag in ['a', 'r', 'n', 'v']:
        lemma_word = stemmer.lemmatize(word, main_tag)
      else:
        lemma_word = word

      lemma_word_list.append(lemma_word)

    doc = ' '.join([lemma_word for lemma_word in lemma_word_list if len(lemma_word) > 2])
    docs.append(doc)

  vectorizer = CountVectorizer(min_df=0.005, max_df=1.0, stop_words=stopwords.words('english'))
  X_data = vectorizer.fit_transform(docs).toarray()
  tfidf_transformer = TfidfTransformer(sublinear_tf=True)
  X_data = tfidf_transformer.fit_transform(X_data).toarray()

  svd_model = TruncatedSVD(n_components=500)
  X_data = svd_model.fit_transform(X_data)

  y_data = y

  return X_data, y_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [248]:
X_data, y_data = preprocess(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle=True)

## 3. Migrate data to GPU

In [249]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train = torch.Tensor(X_train).to(device)
y_train = torch.Tensor(y_train).to(device).reshape(-1, 1)

X_test = torch.Tensor(X_test).to(device)
y_test = torch.Tensor(y_test).to(device).reshape(-1, 1)

## 4. Define logistic classifier

In [250]:
import torch.nn as nn
import torch.nn.functional as F

class FNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(FNN, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, output_size)

  def forward(self, X):
    out1 = F.relu(self.fc1(X))
    out2 = torch.sigmoid(self.fc2(out1))
    return out2
    

def accuracy(y_pred, y):
  answer = (y_pred >= 0.5).float()
  return torch.mean((answer == y).float())

## 5. Define model training and result showing functions

In [251]:
import numpy as np
import matplotlib.pyplot as plt

def train_model(lr, epoch_count, layer_size_list, lamb):
  fnn = FNN(*layer_size_list).to(device)
  criterion = nn.BCELoss()
  optimizer = torch.optim.SGD(fnn.parameters(), lr=lr, weight_decay=lamb)

  history = {
      'train_loss': [],
      'test_loss': [],
      'train_acc': [],
      'test_acc': []
  }

  for epoch in range(epoch_count):
    y_train_pred = fnn.forward(X_train)
    train_loss = criterion(y_train_pred, y_train)
    train_acc = accuracy(y_train_pred, y_train)

    y_test_pred = fnn.forward(X_test)
    test_loss = criterion(y_test_pred, y_test)
    test_acc = accuracy(y_test_pred, y_test)

    history['train_loss'].append(train_loss.item())
    history['test_loss'].append(test_loss.item())
    history['train_acc'].append(train_acc.item())
    history['test_acc'].append(test_acc.item())

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

  for key in history.keys():
    history[key] = np.array(history[key])

  return fnn, history

def show_result(history):
  epoch_tick = np.arange(history['train_loss'].shape[0])

  plt.figure(figsize=(12, 8))
  plt.title('Loss curve')
  plt.plot(epoch_tick, history['train_loss'], 'b-', linewidth=5)
  plt.plot(epoch_tick, history['test_loss'], 'r-', linewidth=5)
  plt.legend(['training loss', 'testing loss'], fontsize='x-large', loc='upper right')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.show()

  plt.figure(figsize=(12, 8))
  plt.title('Accuracy curve')
  plt.plot(epoch_tick, history['train_acc'] * 100, 'b-', linewidth=5)
  plt.plot(epoch_tick, history['test_acc'] * 100, 'r-', linewidth=5)
  plt.legend(['training accuracy', 'testing accuracy'], fontsize='x-large', loc='lower right')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.show()

In [252]:
TRAIN_MODE = True
MODEL_SAVE_PATH = 'classifier.pt'

if TRAIN_MODE:
  model, history = train_model(lr=0.3, epoch_count=3000, layer_size_list=(500, 50, 1), lamb=0.005)
  show_result(history)

  torch.save(model.state_dict(), MODEL_SAVE_PATH)
else:
  model = FNN(500, 50, 1).to(device)
  model.load_state_dict(torch.load(MODEL_SAVE_PATH))
  model.eval()

  # forward your test data and check it
  # print(model.forward(X_test))
  # print(y_test)